In [12]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

In [13]:
#url = 'https://www.remax.ca/ab/calgary-real-estate'
city_name_input = input("Please enter city: ")
city_name = city_name_input.lower()
url = f'https://www.remax.ca/ab/{city_name}-real-estate'

Please enter city: Calgary


In [14]:
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")
#print(soup.prettify())

In [15]:
def parse(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    df = pd.DataFrame(columns=["Bed","Bath","SQFT","Type"])
    for each in soup.find_all(class_= "card-text ng-star-inserted" ):
        try: 
            title = each.find(class_='price').text.replace('\n', '')
        except:
            title = 'None'
        try:
            location = each.find(class_="address ng-star-inserted").text.replace('\n', '')
        except:
            location = 'None'
        try: 
            company = each.find(class_='mlsNumber').text.replace('\n', '')
        except:
            company = 'None'
        try:
            salary = each.find(class_='content-bottom is-flex ng-star-inserted').text
        except:
            salary = 'None'
        #synopsis = each.find('span', {'class':'summary'}).text.replace('\n', '')
        df = df.append({'Bed':title, 'Bath':location, 'SQFT':company, 'Type':salary}, ignore_index=True)
    return df

In [16]:
a = url + '?page='
#url_dump = []
#for i in range(148):
data_dump = pd.DataFrame([])
for i in range(1,2):
    time.sleep(5)
    url_dump1 = a+(f'{i:1}')
    df_dump1 = parse(url_dump1)
    data_dump = data_dump.append(df_dump1)
data_dump.head()

,Bed,Bath,SQFT,Type
0,"$380,888","122 26 AVE NW, Calgary, AB, T2M 2C7",MLS® #: A1022334,2 bed|1 + 1 bath|1159 sqft|house
1,"$480,000","21 KENTISH DR SW, Calgary, AB, T2V 2L4",MLS® #: A1022501,4 bed|2 bath|1091 sqft|house
2,"$465,000","1408 NORTHMOUNT DR NW, Calgary, AB, T2L 0G2",MLS® #: A1021053,3 bed|2 bath|1046 sqft|house
3,"$365,000","275 TARALAKE TERR NE, Calgary, AB, T3J 0A1",MLS® #: A1021788,4 bed|2 + 1 bath|1263 sqft|house
4,"$599,900","1040 MEMORIAL DR NW, Calgary, AB, T2N 3E1",MLS® #: A1022373,3 bed|2 + 2 bath|1305 sqft|house


In [17]:
data1a = data_dump.replace('N/A sqft',np.nan, regex=True)
data1a.head()

,Bed,Bath,SQFT,Type
0,"$380,888","122 26 AVE NW, Calgary, AB, T2M 2C7",MLS® #: A1022334,2 bed|1 + 1 bath|1159 sqft|house
1,"$480,000","21 KENTISH DR SW, Calgary, AB, T2V 2L4",MLS® #: A1022501,4 bed|2 bath|1091 sqft|house
2,"$465,000","1408 NORTHMOUNT DR NW, Calgary, AB, T2L 0G2",MLS® #: A1021053,3 bed|2 bath|1046 sqft|house
3,"$365,000","275 TARALAKE TERR NE, Calgary, AB, T3J 0A1",MLS® #: A1021788,4 bed|2 + 1 bath|1263 sqft|house
4,"$599,900","1040 MEMORIAL DR NW, Calgary, AB, T2N 3E1",MLS® #: A1022373,3 bed|2 + 2 bath|1305 sqft|house


In [18]:
data2 = data1a.dropna()
data2.head()

,Bed,Bath,SQFT,Type
0,"$380,888","122 26 AVE NW, Calgary, AB, T2M 2C7",MLS® #: A1022334,2 bed|1 + 1 bath|1159 sqft|house
1,"$480,000","21 KENTISH DR SW, Calgary, AB, T2V 2L4",MLS® #: A1022501,4 bed|2 bath|1091 sqft|house
2,"$465,000","1408 NORTHMOUNT DR NW, Calgary, AB, T2L 0G2",MLS® #: A1021053,3 bed|2 bath|1046 sqft|house
3,"$365,000","275 TARALAKE TERR NE, Calgary, AB, T3J 0A1",MLS® #: A1021788,4 bed|2 + 1 bath|1263 sqft|house
4,"$599,900","1040 MEMORIAL DR NW, Calgary, AB, T2N 3E1",MLS® #: A1022373,3 bed|2 + 2 bath|1305 sqft|house


In [19]:
data3 = data2['Type'].str.split('|', expand=True)
data3.head()

,0,1,2,3
0,2 bed,1 + 1 bath,1159 sqft,house
1,4 bed,2 bath,1091 sqft,house
2,3 bed,2 bath,1046 sqft,house
3,4 bed,2 + 1 bath,1263 sqft,house
4,3 bed,2 + 2 bath,1305 sqft,house


In [20]:
data3_adr = data2['Bath'].str.split(',', expand=True)
data3_adr.head()

,0,1,2,3
0,122 26 AVE NW,Calgary,AB,T2M 2C7
1,21 KENTISH DR SW,Calgary,AB,T2V 2L4
2,1408 NORTHMOUNT DR NW,Calgary,AB,T2L 0G2
3,275 TARALAKE TERR NE,Calgary,AB,T3J 0A1
4,1040 MEMORIAL DR NW,Calgary,AB,T2N 3E1


In [21]:
data_dump1_test = data_dump1['Bath'].str.split(', Calgary, AB, ', expand=True).copy()
data_dump1_test.head()

,0,1
0,122 26 AVE NW,T2M 2C7
1,21 KENTISH DR SW,T2V 2L4
2,1408 NORTHMOUNT DR NW,T2L 0G2
3,275 TARALAKE TERR NE,T3J 0A1
4,1040 MEMORIAL DR NW,T2N 3E1


In [42]:
data4 = data2.replace('MLS® #: ','', regex=True)
data4.head()

,Bed,Bath,SQFT,Type
0,"$1,625,000","411 PATTERSON BLVD SW, Calgary, AB, T3H 3N6",C4176302,4 bed|3.1 bath|3724 sqft|house
1,"$550,000","2635 34 ST SW, Calgary, AB, T3E 2W6",C4170887,3 bed|2 bath|1095 sqft|house
5,"$589,900","278 MASTERS AVE SE, Calgary, AB, T3M 2C1",C4162838,4 bed|3.1 bath|2033 sqft|house
6,"$1,200,000","1920 12 AVE SW, Sunalta, Calgary, AB, T3C 0R8",C4161758,3 bed|8 bath|2241 sqft|house
7,"$379,000","32 MARTIN CROSSING CRES NE, Martindale, Calgar...",C4142534,4 bed|3 bath|1438 sqft|house


In [43]:
data4.columns = ['Bed_1', 'Address', 'MLS', 'Details']

In [44]:
data5 = data2['Bed'].str.replace('$','')
data5.head()

0     1,625,000 
1       550,000 
5       589,900 
6     1,200,000 
7       379,000 
Name: Bed, dtype: object

In [45]:
all_data_1 = pd.concat([data5, data4, data3], axis=1, sort=False)
all_data_1.head()

,Bed,Bed_1,Address,MLS,Details,0,1,2,3
0,"1,625,000","$1,625,000","411 PATTERSON BLVD SW, Calgary, AB, T3H 3N6",C4176302,4 bed|3.1 bath|3724 sqft|house,4 bed,3.1 bath,3724 sqft,house
1,"550,000","$550,000","2635 34 ST SW, Calgary, AB, T3E 2W6",C4170887,3 bed|2 bath|1095 sqft|house,3 bed,2 bath,1095 sqft,house
5,"589,900","$589,900","278 MASTERS AVE SE, Calgary, AB, T3M 2C1",C4162838,4 bed|3.1 bath|2033 sqft|house,4 bed,3.1 bath,2033 sqft,house
6,"1,200,000","$1,200,000","1920 12 AVE SW, Sunalta, Calgary, AB, T3C 0R8",C4161758,3 bed|8 bath|2241 sqft|house,3 bed,8 bath,2241 sqft,house
7,"379,000","$379,000","32 MARTIN CROSSING CRES NE, Martindale, Calgar...",C4142534,4 bed|3 bath|1438 sqft|house,4 bed,3 bath,1438 sqft,house


In [46]:
all_data_2 = all_data_1.drop(['Bed_1','Details'], axis=1)
all_data_2.head()

,Bed,Address,MLS,0,1,2,3
0,"1,625,000","411 PATTERSON BLVD SW, Calgary, AB, T3H 3N6",C4176302,4 bed,3.1 bath,3724 sqft,house
1,"550,000","2635 34 ST SW, Calgary, AB, T3E 2W6",C4170887,3 bed,2 bath,1095 sqft,house
5,"589,900","278 MASTERS AVE SE, Calgary, AB, T3M 2C1",C4162838,4 bed,3.1 bath,2033 sqft,house
6,"1,200,000","1920 12 AVE SW, Sunalta, Calgary, AB, T3C 0R8",C4161758,3 bed,8 bath,2241 sqft,house
7,"379,000","32 MARTIN CROSSING CRES NE, Martindale, Calgar...",C4142534,4 bed,3 bath,1438 sqft,house


In [47]:
all_data_2.columns = ['Cost', 'Address', 'MLS', 'Bedrooms', 'Bathrooms', 'Area', 'Type']
all_data_2.head()

,Cost,Address,MLS,Bedrooms,Bathrooms,Area,Type
0,"1,625,000","411 PATTERSON BLVD SW, Calgary, AB, T3H 3N6",C4176302,4 bed,3.1 bath,3724 sqft,house
1,"550,000","2635 34 ST SW, Calgary, AB, T3E 2W6",C4170887,3 bed,2 bath,1095 sqft,house
5,"589,900","278 MASTERS AVE SE, Calgary, AB, T3M 2C1",C4162838,4 bed,3.1 bath,2033 sqft,house
6,"1,200,000","1920 12 AVE SW, Sunalta, Calgary, AB, T3C 0R8",C4161758,3 bed,8 bath,2241 sqft,house
7,"379,000","32 MARTIN CROSSING CRES NE, Martindale, Calgar...",C4142534,4 bed,3 bath,1438 sqft,house


In [48]:
all_data_3 = all_data_2.replace(' bed','', regex=True)
all_data_4 = all_data_3.replace(' bath','', regex=True)
all_data_5 = all_data_4.replace(' sqft','', regex=True)
all_data_5.head()

,Cost,Address,MLS,Bedrooms,Bathrooms,Area,Type
0,"1,625,000","411 PATTERSON BLVD SW, Calgary, AB, T3H 3N6",C4176302,4,3.1,3724,house
1,"550,000","2635 34 ST SW, Calgary, AB, T3E 2W6",C4170887,3,2,1095,house
5,"589,900","278 MASTERS AVE SE, Calgary, AB, T3M 2C1",C4162838,4,3.1,2033,house
6,"1,200,000","1920 12 AVE SW, Sunalta, Calgary, AB, T3C 0R8",C4161758,3,8,2241,house
7,"379,000","32 MARTIN CROSSING CRES NE, Martindale, Calgar...",C4142534,4,3,1438,house


In [49]:
all_data_5.shape

(870, 7)

In [51]:
all_data_5.Type.value_counts()

house        504
condo        230
townhouse    130
Name: Type, dtype: int64

In [35]:
all_data_5.to_csv('CalgaryRealEstate.csv')

In [26]:
import numpy as np
import pandas as pd
import pyodbc
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-Q1QKHGM\SQLEXPRESS;'
                      'Database=DSTRAINING;'
                      'Trusted_Connection=yes;')

In [29]:
selectQuery = 'SELECT * FROM DSTRAINING.[dbo].[WRK_CalgaryRealEstate_20190924]'
df = pd.read_sql(selectQuery, conn)
df.head()

,RowNumber,Cost,Address,MLS,Bedrooms,Bathrooms,Area,Type
0,1,1625000,"411 PATTERSON BLVD SW, Calgary, AB, T3H 3N6",C4176302,4,3.1,3724,house
1,2,550000,"2635 34 ST SW, Calgary, AB, T3E 2W6",C4170887,3,2.0,1095,house
2,3,589900,"278 MASTERS AVE SE, Calgary, AB, T3M 2C1",C4162838,4,3.1,2033,house
3,4,1200000,"1920 12 AVE SW, Sunalta, Calgary, AB, T3C 0R8",C4161758,3,8.0,2241,house
4,5,379000,"32 MARTIN CROSSING CRES NE, Martindale, Calgar...",C4142534,4,3.0,1438,house


In [30]:
df.describe()

,RowNumber,Cost,Bedrooms,Bathrooms,Area
count,680.000000,6.800000e+02,680.000000,680.000000,680.000000
mean,340.500000,7.027997e+05,3.108824,2.327647,1620.970588
std,196.443376,7.459890e+05,1.365441,1.054816,875.713186
min,1.000000,9.975000e+04,0.000000,0.100000,338.000000
25%,170.750000,3.200000e+05,2.000000,2.000000,1002.500000
50%,340.500000,4.999000e+05,3.000000,2.100000,1410.500000
75%,510.250000,7.880000e+05,4.000000,3.100000,2049.000000
max,680.000000,7.950000e+06,11.000000,8.000000,5327.000000


----

### **Testing**

In [28]:
url_1 = 'https://www.remax.ca/ab/calgary-real-estate/720-37-st-nw-wp_id251536557-lst'
def parse_1(url_1):
    html = requests.get(url_1)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    df_1 = pd.DataFrame(columns=['a1', 'a2', 'a3'])
    for each in soup.find_all('section', {'class' : 'property-details'} ):
        try: 
            a1 = each.find('div', {'class' : 'detail'}).text.replace('\n', '')
        except:
            a1 = 'None'
        try: 
            a2 = each.find('div', {'class' : 'collapsible-details'}).text.replace('\n', '')
        except:
            a2 = 'None'
        try: 
            a3 = each.find('div', {'class' : 'detail-wrapper ng-star-inserted'}).text.replace('\n', '')
        except:
            a3 = 'None'
       
        df_1 = df_1.append({'a1':a1, 'a2':a2, 'a3':a3}, ignore_index=True)
    return df_1
df_1 = parse_1(url_1)
df_1

,a1,a2,a3
0,DetailsProperty Type: DuplexProperty Tax: N/AL...,InsideHeating: Forced Air-1Heating Source: Nat...,None


In [15]:
#df_1.to_csv('test.csv')

https://www.walkscore.com/score/t2m%202c7